# Ejercicio 1

In [1]:
import numpy as np

def RadioEspectral(A):
    """
    Esta función calcula el radio espectral de una matriz A
    """
    # Hallamos los valores propios
    valores, vectores = np.linalg.eig(A)
    # Hallamos el valor propio más grande
    rho = max(abs(valores))
    return rho

In [2]:
import numpy as np

# Ejercicio 1.

# Creamos la matriz D
D = 8*np.identity(10)
# Creamos la matriz M
M = np.identity(10)
for i in range(len(M)-2):
    for j in range(i,i+1):
        M[i,j+1] = -1
        M[i,j+2] = -1
M[len(M)-2, len(M)-1] = -1
# Creamos la matriz N
N = np.transpose(M)
# Definimos las matrices de iteración para cada método iterativo 
M1 = np.dot(np.linalg.inv(M+D), N)
M2 = np.dot(np.linalg.inv(D), M+N)
M3 = np.dot(np.linalg.inv(M+N), D)
# Hallamos el radio espectral de cada matriz de iteración
rho1 = RadioEspectral(M1)
rho2 = RadioEspectral(M2)
rho3 = RadioEspectral(M3)
#Impresión de resultados
print("El radio espectral para el primer método iterativo es {}".format(rho1))
print("El radio espectral para el segundo método iterativo es {}".format(rho2))
print("El radio espectral para el tercer método iterativo es {}".format(rho3))

El radio espectral para el primer método iterativo es 0.14503954977535374
El radio espectral para el segundo método iterativo es 0.5000000000000001
El radio espectral para el tercer método iterativo es 12.287023474955681


# Ejercicio 2

In [3]:
def MatrizJacobi(A):
    """
    Esta función calcula la matriz de iteración del método
    de Jacobi, definido por la descomposición M=D y N=D-A
    donde D=(a11, . . ., ann)
    """
    n = len(A)
    D = np.zeros((n,n))
    for i in range(n):
        D[i][i] = A[i][i]
    M = D
    N = D - A
    J = np.dot(np.linalg.inv(M),N)
    return J

In [4]:
def MatrizGaussSeidel(A):
    """
    Esta función calcula la matriz de iteración del método
    de Gauss-Seidel, definido por la descomposición M=D-E
    y N=F
    """
    n = len(A)
    D = np.zeros((n,n))
    E = np.zeros((n,n))
    F = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i == j:
                D[i][j] = A[i][j]
            elif i > j :
                E[i][j] = - A[i][j]
            else:
                F[i][j] = - A[i][j]
    M = D - E
    N = np.copy(F)
    G = np.dot(np.linalg.inv(M), N)
    return G   

In [5]:
def MatrizRichardson(A, P, alpha, optimo = True):
    """
    Esta función calcula la matriz de iteración del método de
    Richardson:  R = I - P^(-1) A 
    
    Argumentos:
    A: matriz del sistema Ax = b que se desea resolver
    P: matriz no singular, precondicionador de A
    alpha: número real positivo
    optimo: valor booleano  para usar el alpha óptimo
    
    Salida:
    alpha: valor óptimo de alpha si óptimo es verdero
    R: Matriz de iteración
    """
    n = len(A)
    # Hallamoas los valores porpios de la matriz P^(-1)A
    invPA = np.dot(np.linalg.inv(P), A)
    valores, vectores = np.linalg.eig(invPA)
    lambdamin = min(valores)
    lambdamax = max(valores)
    if optimo == True:
        alphaopt = 2/(lambdamin + lambdamax)
        alpha = alphaopt
    I = np.identity(n)
    R = I - alpha * np.dot(np.linalg.inv(P), A)
    return (alpha, R)

In [6]:
def Jacobi(A, b, x0, tol, iterMax):
    """
    Esta función caula la solución al sistema Ax=b
    mediante el método de Jacobi.
    
    Agumentos:
    A: (matriz cuadrada): Matriz cuadrada invertible
    b: vector unidimensional de valores independientes
    x0: Solución inicial
    tol: Tolerancia
    iterMax: Número máximo de iteraciones
    
    Salida
    (Iter, x ): Tupla que contiene el número de iteraciones
                y la solución aproximada  del sistema
    """
    # Creamos el vector solución x y definimos las iteraciones Iter
    n = len(A)
    Iter = 1
    x = np.empty(n)
    # El siguiente ciclo termina si se supera el número máximo de iteraciones
    while Iter <= iterMax:
        for i in range(n):
            suma = 0
            for j in range(n):
                if j != i:
                    suma += A[i][j] * x0[j]
            # Calculamos la solución en la iteración Iter
            x[i] = (b[i] - suma)/ A[i][i]
        # Calculamos el error cometido en la iteración
        error = np.linalg.norm(b - np.dot(A,x)) / np.linalg.norm(b)
        # Verificamos si el error es menor que la tolerancia dada
        if error < tol:
            return(Iter,x)
        # Incrementamos el número de iteraciones
        Iter += 1
        # Actualizamos la solución
        x0 = x.copy()
    return (Iter, x)

In [7]:
def GaussSeidel(A, b, x0, tol, iterMax):
    """
    Esta función calcula la solución al sistema Ax = b mediante
    el método de Gauss-Seidel
    
    Argumentos:
    A : Matriz cuadrada del sistema
    b : vector unidimensional de valores independientes
    x0 : Solución inicial
    tol: Tolerancia
    iterMax: Número máximo de iteraciones
    
    Salida:
    (Iter, x ): Tupla que contiene el número de iteraciones
                y la solución aproximada  del sistema
    """
    # Creamos el vector solución x
    n = len(A)
    x = np.empty(n)
    # Inicializamos el número de iteraciones
    Iter = 1
    # Realizamos las iteraciones necesarias para 
    while Iter <= iterMax:
        for i in range(n):
            suma1, suma2 = 0,0
            for j in range(i):
                suma1 += A[i][j] * x[j]
            for j in range(i + 1, n):
                suma2 += A[i][j] * x[j]
            # Calculamos la solución en la iteración Iter
            x[i] = (b[i] - suma1 - suma2)/A[i][i]
        # Calculamos el error cometido en la iteración 
        error = np.linalg.norm(b - np.dot(A,x)) / np.linalg.norm(b)
        #Verificamos si el error es menor que la tolerancia dada
        if error < tol:
            return(Iter, x)
        # Actualizamos el número de iteraciones
        Iter += 1
        # Actualizamos la solución
        x0 = x.copy()
    return(Iter, x)

In [8]:
def Richardson(A, b, P, x0, tol, iterMax, alpha, optimo = True):
    """
    Esta función calcula la solución al sistema Ax = b mediante
    el método de Richardson 
    """
    # Inicializamos el contador de iteraciones k
    k = 0
    n = len(A)
    normab = np.linalg.norm(b)
    # Creamos el vector solución
    x = np.empty(n)
    # Hallamos el valor alpha óptimo
    if optimo:
        alpha = MatrizRichardson(A, P, alpha, optimo = True)[0]
    # Calculamos el residual inicial
    r = b - np.dot(A,x0)
    # Hallamos la inversa del precondicionador
    invP = np.linalg.inv(P)
    while k <= iterMax:
        y = np.dot(invP, r)
        # Calculamos el valor de la solución
        x = x0 + alpha * y
        # Actualizamos el valor del residual de la késima iteración
        r = r - alpha * np.dot(A,y)
        # Calculamos el error
        error = np.linalg.norm(b - np.dot(A,x))/normab
        if error < tol:
            return (k,x)
        # Actualizamos la solución
        k += 1
        x0 = x.copy()
    return (k, x)

In [9]:
import numpy as np

# Ejercicio 2.

# Definimos la matriz A
A = np.array([
    [62,24,1,8,15],
    [23,50,7,14,16],
    [4,6,58,20,22],
    [10,12,19,66,3],
    [11,18,25,2,54]
    ])
# Definimos el vector b
b = np.array([110,110,110,110,110])

# Inciso (1)

# Definimos la solución inicial, tolerancia y número máximo de iteraciones
x0 = np.zeros(len(A))
tol = 1e-15
iterMax = 100
# Calculamos el radio espectral de cada método
J = MatrizJacobi(A)
G = MatrizGaussSeidel(A)
rhoj = RadioEspectral(J)
rhogs = RadioEspectral(G)
# Hallamos la solución mediante el método de Jacobi y Gauss-Seidel
(Iter1, SolJacobi) = Jacobi(A, b, x0, tol, iterMax)
(Iter2, SolGS) = GaussSeidel(A, b, x0, tol, iterMax)
# Impresión de resultados 
print("Para el método de Jacobi se tienen los siguientes resultados:")
print("Radio espectral = {}".format(rhoj))
print("Número de iteraciones = {}".format(Iter1))
print("Solución = {}".format(SolJacobi))
print("\n")
print("Para el método de Gauss-Seidel se tienen los siguientes resultados:")
print("Radio espectral = {}".format(rhogs))
print("Número de iteraciones = {}".format(Iter2))
print("Solución = {}".format(SolGS))
print("\n")

# Inciso (2)
print("Resultados cuando se usa el método de Richardson \n")

n = len(A)

# Si P = I
P = np.identity(n)
# Calculamos el radio espectral asociado al alpha óptimo
alphaopt1, R1 = MatrizRichardson(A, P, 1, optimo = True)
rho1 = RadioEspectral(R1)
# Hallamos la solución del sistema
iter1, sol1 =  Richardson(A, b, P, x0, tol, iterMax, 1, optimo = True)
# Impresión de resultados
print("Si P = I se tiene que:")
print("El alpha óptimo es {}".format(alphaopt1))
print("El radio espectral mínimo es {}".format(rho1))
print("La solución es {} en {} iteraciones".format(sol1, iter1))
print("\n")

# Si P = D
D = np.zeros((n,n))
for i in range(n):
    D[i][i] = A[i][i]
P = D
# Calculamos el radio espectral asociado al alpha óptimo
alphaopt2, R2 = MatrizRichardson(A, P, 1, optimo = True)
rho2 = RadioEspectral(R2)
# Hallamos la solución del sistema
iter2, sol2 =  Richardson(A, b, P, x0, tol, iterMax, 1, optimo = True)
# Impresión de resultados
print("Si P = D se tiene que:")
print("El alpha óptimo es {}".format(alphaopt2))
print("El radio espectral mínimo es {}".format(rho2))
print("La solución es {} en {} iteraciones".format(sol1, iter2))
print("\n")

Para el método de Jacobi se tienen los siguientes resultados:
Radio espectral = 0.927984216764368
Número de iteraciones = 101
Solución = [0.99947837 0.99931314 0.99946658 0.99958324 0.99935602]


Para el método de Gauss-Seidel se tienen los siguientes resultados:
Radio espectral = 0.30657833775344734
Número de iteraciones = 23
Solución = [1. 1. 1. 1. 1.]


Resultados cuando se usa el método de Richardson 

Si P = I se tiene que:
El alpha óptimo es 0.01495626401089898
El radio espectral mínimo es 0.6451890411988882
La solución es [1. 1. 1. 1. 1.] en 78 iteraciones


Si P = D se tiene que:
El alpha óptimo es 0.8509810205973402
El radio espectral mínimo es 0.6406779764777074
La solución es [1. 1. 1. 1. 1.] en 77 iteraciones




# Ejercicio 3

In [10]:
def MetPotencia(A, x0, tol, itermax):
    """
    Esta función calcula el valor propio dominante
    (valor propio con magnitud más grande) de la matriz A
    mediante el método de la potencia
    
    Argumentos:
    A: Matriz cuadrada de la que sea desea hallar el valor propio
    x0: Valor inicial
    tol: Tolerancia
    itermax:  Número máximo de iteraciones
    
    Salida:
    (Lambda, k): Tupla que contiene el valor proipio buscado
                 y el número de iteraciones realizados
    """
    k = 1
    n = len(A)
    x =  x0 / np.linalg.norm(x0)
    y = np.dot(A,x0)
    while k <= itermax:
        x = y/np.linalg.norm(y)
        y = np.dot(A,x)
        Lambda = np.dot(np.transpose(x), y)
        error = np.linalg.norm(np.dot(A,x) - Lambda*x)
        if error < tol:
            return (Lambda, k)
        k += 1
    return (Lambda, k)
        

In [21]:
import math

# Ejercicio 3.

# Definimos la matriz A

A = np.array([[1, -1, 2],
             [-2, 0, 5],
             [6, -3, 6]])
n = len(A)
tol = 1e-3
itermax = 50

# Cuando x0 =  1 / sqrt(3)
x0 = (1/math.sqrt(3)) * np.ones((3,))
Lambda, Iter = MetPotencia(A, x0, tol, itermax)
print("Tomando en cuenta q0 = 1/sqrt(3), se obtiene")
print("El valor propio más grande es {}".format(Lambda))
print("Número de iteraciones {}".format(Iter))
print("\n")

# Si x0 = w0 / norm(w0)
valores, vectores = np.linalg.eig(A)
v2 = vectores[1]
v3 = vectores[2]
w0 = (1/3)*v2 - (2/3)*v3
x0 = w0 / np.linalg.norm(w0)
Lambda, Iter = MetPotencia(A, x0, tol, itermax)
print("Tomando en cuenta q0 = w0/norm(w0), se obtiene")
print("El valor propio más grande es {}".format(Lambda))
print("Número de iteraciones {}".format(Iter))

Tomando en cuenta q0 = 1/sqrt(3), se obtiene
El valor propio más grande es 5.002243161396065
Número de iteraciones 13


Tomando en cuenta q0 = w0/norm(w0), se obtiene
El valor propio más grande es 5.0036318738009165
Número de iteraciones 13
